In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv('fetal_health.csv')

# EDA and Preprocessing

In [8]:
print(df.isnull().sum())

baseline value                                            0
accelerations                                             0
fetal_movement                                            0
uterine_contractions                                      0
light_decelerations                                       0
severe_decelerations                                      0
prolongued_decelerations                                  0
abnormal_short_term_variability                           0
mean_value_of_short_term_variability                      0
percentage_of_time_with_abnormal_long_term_variability    0
mean_value_of_long_term_variability                       0
histogram_width                                           0
histogram_min                                             0
histogram_max                                             0
histogram_number_of_peaks                                 0
histogram_number_of_zeroes                                0
histogram_mode                          

In [9]:
print(df['fetal_health'].value_counts())

fetal_health
1.0    1655
2.0     295
3.0     176
Name: count, dtype: int64


**Unbalanced classes with high probabilistic output for Class 1, so I am going to use Naive-Bayes Classifier**

**Use Min-Max Scaler so all data between [0,1]**

In [10]:
from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()

feature_df = df.drop(columns='fetal_health', axis=1)

normalized_feature_df = pd.DataFrame(normalizer.fit_transform(feature_df), columns=feature_df.columns)

normalized_feature_df.reset_index()

normalized_feature_df.tail()

c:\Users\colby\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\colby\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\colby\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\colby\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning:

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_width,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency
2121,0.629630,0.000000,0.000000,0.466667,0.0,0.0,0.0,0.893333,0.000000,0.274725,...,0.209040,0.798165,0.474138,0.222222,0.0,0.732283,0.706422,0.688073,0.007435,0.5
2122,0.629630,0.052632,0.000000,0.466667,0.0,0.0,0.0,0.880000,0.029412,0.241758,...,0.355932,0.486239,0.405172,0.333333,0.0,0.724409,0.688073,0.678899,0.011152,1.0
2123,0.629630,0.052632,0.000000,0.466667,0.0,0.0,0.0,0.893333,0.029412,0.219780,...,0.361582,0.486239,0.413793,0.277778,0.0,0.732283,0.688073,0.688073,0.014870,1.0
2124,0.629630,0.052632,0.000000,0.400000,0.0,0.0,0.0,0.880000,0.029412,0.296703,...,0.355932,0.486239,0.405172,0.333333,0.0,0.724409,0.678899,0.678899,0.014870,1.0
2125,0.666667,0.105263,0.004158,0.533333,0.0,0.0,0.0,0.826667,0.029412,0.395604,...,0.220339,0.614679,0.318966,0.111111,0.1,0.669291,0.642202,0.623853,0.003717,0.5


# Naive Bayes Classifier Model

In [27]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB, GaussianNB
from sklearn.model_selection import train_test_split, cross_val_score

health_features = normalized_feature_df.values

health_result = df['fetal_health'].values

classifier_list=[MultinomialNB(), ComplementNB(), GaussianNB()]

scoring_list = ['weighted', 'micro', 'macro']

for clf in classifier_list:

    name = clf.__class__.__name__
    
    X_training_val, X_testing, y_training_val, y_testing = train_test_split(health_features, health_result, test_size=0.25)

    clf.fit(X_training_val, y_training_val)
    
    for score in scoring_list:
        
        f1_list = []

        f1_scores = cross_val_score(clf, X_testing, y_testing, cv=10, scoring=f'f1_{score}')

        f1_list.append((f1_scores).mean())

        print(f'{name} Classifier {score.capitalize()} F1 Score:',max(f1_list).round(3))
       


MultinomialNB Classifier Weighted F1 Score: 0.759
MultinomialNB Classifier Micro F1 Score: 0.819
MultinomialNB Classifier Macro F1 Score: 0.424
ComplementNB Classifier Weighted F1 Score: 0.823
ComplementNB Classifier Micro F1 Score: 0.808
ComplementNB Classifier Macro F1 Score: 0.665
GaussianNB Classifier Weighted F1 Score: 0.708
GaussianNB Classifier Micro F1 Score: 0.675
GaussianNB Classifier Macro F1 Score: 0.593
